In [2]:
import re
from datetime import date

import math
import pandas as pd

In [2]:
# '''
# --------------------- Copyright Block ----------------------
# praytimes.py: Prayer Times Calculator (ver 2.3)
# Copyright (C) 2007-2011 PrayTimes.org
# Python Code: Saleem Shafi, Hamid Zarrabi-Zadeh
# Original js Code: Hamid Zarrabi-Zadeh
# License: GNU LGPL v3.0
# TERMS OF USE:
# 	Permission is granted to use this code, with or
# 	without modification, in any website or application
# 	provided that credit is given to the original work
# 	with a link back to PrayTimes.org.
# This program is distributed in the hope that it will
# be useful, but WITHOUT ANY WARRANTY.
# PLEASE DO NOT REMOVE THIS COPYRIGHT BLOCK.
# '''

# --------------------- Help and Manual ----------------------
# User's Manual:
# http://praytimes.org/manual
# Calculation Formulas:
# http://praytimes.org/calculation

# ----------------------- User Interface -------------------------
# 	getTimes (date, coordinates, timeZone [, dst [, timeFormat]])
# 	setMethod (method)       // set calculation method
# 	adjust (parameters)      // adjust calculation parameters
# 	tune (offsets)           // tune times by given offsets
# 	getMethod ()             // get calculation method
# 	getSetting ()            // get current calculation parameters
# 	getOffsets ()            // get current time offsets

# ------------------------- Sample Usage --------------------------
# PT = PrayTimes('ISNA')
# times = PT.getTimes((2011, 2, 9), (43, -80), -5)
# times['sunrise']
# 07:26

In [3]:
#----------------------- PrayTimes Class ------------------------

class PrayTimes():


    #------------------------ Constants --------------------------

    # Time Names
    timeNames = {
        'imsak'    : 'Imsak',
        'fajr'     : 'Fajr',
        'sunrise'  : 'Sunrise',
        'dhuhr'    : 'Dhuhr',
        'asr'      : 'Asr',
        'sunset'   : 'Sunset',
        'maghrib'  : 'Maghrib',
        'isha'     : 'Isha',
        'midnight' : 'Midnight'
    }

    # Calculation Methods
    methods = {
        'MWL': {
            'name': 'Muslim World League',
            'params': { 'fajr': 18, 'isha': 17 } },
        'ISNA': {
            'name': 'Islamic Society of North America (ISNA)',
            'params': { 'fajr': 15, 'isha': 15 } },
        'Egypt': {
            'name': 'Egyptian General Authority of Survey',
            'params': { 'fajr': 19.5, 'isha': 17.5 } },
        'Makkah': {
            'name': 'Umm Al-Qura University, Makkah',
            'params': { 'fajr': 18.5, 'isha': '90 min' } },  # fajr was 19 degrees before 1430 hijri
        'Karachi': {
            'name': 'University of Islamic Sciences, Karachi',
            'params': { 'fajr': 18, 'isha': 18 } },
        'Kuwait': {
            'name': 'Ministry of Awqaf and Islamic Affairs,Kuwait',
            'params': { 'fajr': 18, 'isha': 17.5 } },
        'France': {
            'name': 'Union des organisations islamiques de France ',
            'params': { 'fajr': 12, 'isha': 12 } },
        'Tehran': {
            'name': 'Institute of Geophysics, University of Tehran',
            'params': { 'fajr': 17.7, 'isha': 14, 'maghrib': 4.5, 'midnight': 'Jafari' } },  # isha is not explicitly specified in this method
        'Jafari': {
            'name': 'Shia Ithna-Ashari, Leva Institute, Qum',
            'params': { 'fajr': 16, 'isha': 14, 'maghrib': 4, 'midnight': 'Jafari' } }
    }

    # Default Parameters in Calculation Methods
    defaultParams = {
        'maghrib': '0 min', 'midnight': 'Standard'
    }


    #---------------------- Default Settings --------------------

    calcMethod = 'MWL'

    # do not change anything here; use adjust method instead
    settings = {
        "imsak"    : '10 min',
        "dhuhr"    : '0 min',
        "asr"      : 'Standard',
        "highLats" : 'None'#Err# 'NightMiddle'
    }

    timeFormat = '24h'
    timeSuffixes = ['am', 'pm']
    invalidTime =  '-----'

    numIterations = 1
    offset = {}


    #---------------------- Initialization -----------------------

    def __init__(self, method = "MWL") :

        # set methods defaults
        for method, config in self.methods.items():
            for name, value in self.defaultParams.items():
                if not name in config['params'] or config['params'][name] is None:
                    config['params'][name] = value

        # initialize settings
        self.calcMethod = method if method in self.methods else 'MWL'
        params = self.methods[self.calcMethod]['params']
        for name, value in params.items():
            self.settings[name] = value

        # init time offsets
        for name in self.timeNames:
            self.offset[name] = 0


    #-------------------- Interface Functions --------------------

    def setMethod(self, method):
        if method in self.methods:
            #err# self.adjust(self.methods[method].params)
            self.adjust(self.methods[method]['params'])
            self.calcMethod = method

    def adjust(self, params):
        self.settings.update(params)

    def tune(self, timeOffsets):
        self.offsets.update(timeOffsets)

    def getMethod(self):
        return self.calcMethod

    def getSettings(self):
        return self.settings

    def getOffsets(self):
        return self.offset

    def getDefaults(self):
        return self.methods

    # return prayer times for a given date
    def getTimes(self, date, coords, timezone, dst = 0, is_formatted=True, formatting=None):
        #def getTimes(self, date, coords, timezone, dst=0, formatting=None):

        self.lat = coords[0]

        #Added to adjust higher latitudes
        if self.lat>=55:
            self.settings["highLats"]= 'NightMiddle'
        # else:
        #     self.settings["highLats"]= 'None'

        self.lng = coords[1]
        self.elv = coords[2] if len(coords)>2 else 0

        #Modified
        self.is_formatted = is_formatted
        if formatting != None:
            self.timeFormat = formatting

        if type(date).__name__ == 'date':
            date = (date.year, date.month, date.day)
        self.timeZone = timezone + (1 if dst else 0)
        self.jDate = self.julian(date[0], date[1], date[2]) - self.lng / (15 * 24.0)
        return self.computeTimes()

    # convert float time to the given format (see timeFormats)
    def getFormattedTime(self, time, formatting, suffixes = None):
        if math.isnan(time):
            return self.invalidTime
        if not self.is_formatted:
            return time
        if formatting.lower() == 'Float'.lower():
            return time
        if suffixes == None:
            suffixes = self.timeSuffixes

        time = self.fixhour(time+ 0.5/ 60)  # add 0.5 minutes to round
        hours = math.floor(time)

        minutes = math.floor((time- hours)* 60)
        suffix = suffixes[ 0 if hours < 12 else 1 ] if formatting == '12h' else ''
        formattedTime = "%02d:%02d" % (hours, minutes) if formatting == "24h" else "%d:%02d" % ((hours+11)%12+1, minutes)
        return formattedTime + suffix


    #---------------------- Calculation Functions -----------------------

    # compute mid-day time
    def midDay(self, time):
        eqt = self.sunPosition(self.jDate + time)[1]
        return self.fixhour(12 - eqt)

    # compute the time at which sun reaches a specific angle below horizon
    def sunAngleTime(self, angle, time, direction = None):
        try:
            decl = self.sunPosition(self.jDate + time)[0]
            noon = self.midDay(time)
            t = 1/15.0* self.arccos((-self.sin(angle)- self.sin(decl)* self.sin(self.lat))/
                    (self.cos(decl)* self.cos(self.lat)))
            return noon+ (-t if direction == 'ccw' else t)
        except ValueError:
            return float('nan')

    # compute asr time
    def asrTime(self, factor, time):
        decl = self.sunPosition(self.jDate + time)[0]
        angle = -self.arccot(factor + self.tan(abs(self.lat - decl)))
        return self.sunAngleTime(angle, time)

    # compute declination angle of sun and equation of time
    # Ref: http://aa.usno.navy.mil/faq/docs/SunApprox.php
    def sunPosition(self, jd):
        D = jd - 2451545.0
        g = self.fixangle(357.529 + 0.98560028* D)
        q = self.fixangle(280.459 + 0.98564736* D)
        L = self.fixangle(q + 1.915* self.sin(g) + 0.020* self.sin(2*g))

        R = 1.00014 - 0.01671*self.cos(g) - 0.00014*self.cos(2*g)
        e = 23.439 - 0.00000036* D

        RA = self.arctan2(self.cos(e)* self.sin(L), self.cos(L))/ 15.0
        eqt = q/15.0 - self.fixhour(RA)
        decl = self.arcsin(self.sin(e)* self.sin(L))

        return (decl, eqt)

    # convert Gregorian date to Julian day
    # Ref: Astronomical Algorithms by Jean Meeus
    def julian(self, year, month, day):
        if month <= 2:
            year -= 1
            month += 12
        A = math.floor(year / 100)
        B = 2 - A + math.floor(A / 4)
        return math.floor(365.25 * (year + 4716)) + math.floor(30.6001 * (month + 1)) + day + B - 1524.5



    #---------------------- Compute Prayer Times -----------------------

    # compute prayer times at given julian date
    def computePrayerTimes(self, times):
        times = self.dayPortion(times)
        params = self.settings

        imsak   = self.sunAngleTime(self.eval(params['imsak']), times['imsak'], 'ccw')
        fajr    = self.sunAngleTime(self.eval(params['fajr']), times['fajr'], 'ccw')
        sunrise = self.sunAngleTime(self.riseSetAngle(self.elv), times['sunrise'], 'ccw')
        dhuhr   = self.midDay(times['dhuhr'])
        asr     = self.asrTime(self.asrFactor(params['asr']), times['asr'])
        sunset  = self.sunAngleTime(self.riseSetAngle(self.elv), times['sunset'])
        maghrib = self.sunAngleTime(self.eval(params['maghrib']), times['maghrib'])
        # print(self.eval(params['isha']), times['isha'], fajr, dhuhr, asr, maghrib)
        isha    = self.sunAngleTime(self.eval(params['isha']), times['isha'])
        return {
            'imsak': imsak, 'fajr': fajr, 'sunrise': sunrise, 'dhuhr': dhuhr,
            'asr': asr, 'sunset': sunset, 'maghrib': maghrib, 'isha': isha
        }

    # compute prayer times
    def computeTimes(self):
        times = {
            'imsak': 5, 'fajr': 5, 'sunrise': 6, 'dhuhr': 12,
            'asr': 13, 'sunset': 18, 'maghrib': 18, 'isha': 18
        }
        # main iterations
        for i in range(self.numIterations):
            times = self.computePrayerTimes(times)
        times = self.adjustTimes(times)
        # add midnight time
        if self.settings['midnight'] == 'Jafari':
            times['midnight'] = times['sunset'] + self.timeDiff(times['sunset'], times['fajr']) / 2
        else:
            times['midnight'] = times['sunset'] + self.timeDiff(times['sunset'], times['sunrise']) / 2

        # Add FastingDuration (Muddat Alsiyam)
        times['fasting'] = times['maghrib'] - times['fajr']
        self.offset['fasting']=0


        times = self.tuneTimes(times)
        return self.modifyFormats(times)

    # adjust times in a prayer time array
    def adjustTimes(self, times):
        params = self.settings
        tzAdjust = self.timeZone - self.lng / 15.0
        for t,v in times.items():
            times[t] += tzAdjust

        if params['highLats'] != 'None':
            times = self.adjustHighLats(times)

        if self.isMin(params['imsak']):
            times['imsak'] = times['fajr'] - self.eval(params['imsak']) / 60.0
        # need to ask about 'min' settings
        if self.isMin(params['maghrib']):
            times['maghrib'] = times['sunset'] - self.eval(params['maghrib']) / 60.0

        if self.isMin(params['isha']):
            # print('ISMIN, MAKKAH method')
            #err# times['isha'] = times['maghrib'] - self.eval(params['isha']) / 60.0
            times['isha'] = times['maghrib'] + self.eval(params['isha']) / 60.0
        times['dhuhr'] += self.eval(params['dhuhr']) / 60.0

        return times

    # get asr shadow factor
    def asrFactor(self, asrParam):
        methods = {'Standard': 1, 'Hanafi': 2}
        return methods[asrParam] if asrParam in methods else self.eval(asrParam)

    # return sun angle for sunset/sunrise
    def riseSetAngle(self, elevation = 0):
        elevation = 0 if elevation == None else elevation
        return 0.833 + 0.0347 * math.sqrt(elevation) # an approximation

    # apply offsets to the times
    def tuneTimes(self, times):
        for name, value in times.items():
            times[name] += self.offset[name] / 60.0
        return times

    # convert times to given time format
    def modifyFormats(self, times):
        for name, value in times.items():
            times[name] = self.getFormattedTime(times[name], self.timeFormat)
            # times[name] = self.getFormattedTime(times[name], self.is_formatted)
        return times

    # adjust times for locations in higher latitudes
    def adjustHighLats(self, times):
        params = self.settings
        nightTime = self.timeDiff(times['sunset'], times['sunrise']) # sunset to sunrise
        times['imsak'] = self.adjustHLTime(times['imsak'], times['sunrise'], self.eval(params['imsak']), nightTime, 'ccw')
        times['fajr']  = self.adjustHLTime(times['fajr'], times['sunrise'], self.eval(params['fajr']), nightTime, 'ccw')
        times['isha']  = self.adjustHLTime(times['isha'], times['sunset'], self.eval(params['isha']), nightTime)
        times['maghrib'] = self.adjustHLTime(times['maghrib'], times['sunset'], self.eval(params['maghrib']), nightTime)
        return times

    # adjust a time for higher latitudes
    def adjustHLTime(self, time, base, angle, night, direction = None):
        portion = self.nightPortion(angle, night)
        diff = self.timeDiff(time, base) if direction == 'ccw' else self.timeDiff(base, time)
        if math.isnan(time) or diff > portion:
            time = base + (-portion if direction == 'ccw' else portion)
        return time

    # the night portion used for adjusting times in higher latitudes
    def nightPortion(self, angle, night):
        method = self.settings['highLats']
        portion = 1/2.0  # midnight
        if method == 'AngleBased':
            portion = 1/60.0 * angle
        if method == 'OneSeventh':
            portion = 1/7.0
        return portion * night

    # convert hours to day portions
    def dayPortion(self, times):
        for i in times:
            times[i] /= 24.0
        return times


    #---------------------- Misc Functions -----------------------

    # compute the difference between two times
    def timeDiff(self, time1, time2):
        return self.fixhour(time2- time1)

    # convert given string into a number
    def eval(self, st):
        val = re.split('[^0-9.+-]', str(st), 1)[0]
        return float(val) if val else 0

    # detect if input contains 'min'
    def isMin(self, arg):
        return isinstance(arg, str) and arg.find('min') > -1


    #----------------- Degree-Based Math Functions -------------------

    def sin(self, d): return math.sin(math.radians(d))
    def cos(self, d): return math.cos(math.radians(d))
    def tan(self, d): return math.tan(math.radians(d))

    def arcsin(self, x): return math.degrees(math.asin(x))
    def arccos(self, x): return math.degrees(math.acos(x))
    def arctan(self, x): return math.degrees(math.atan(x))

    def arccot(self, x): return math.degrees(math.atan(1.0/x))
    def arctan2(self, y, x): return math.degrees(math.atan2(y, x))

    def fixangle(self, angle): return self.fix(angle, 360.0)
    def fixhour(self, hour): return self.fix(hour, 24.0)

    def fix(self, a, mode):
        if math.isnan(a):
            return a
        a = a - mode * (math.floor(a / mode))
        return a + mode if a < 0 else a

In [5]:
#-------------------------- Nesr Code 01 --------------------------
if __name__ == "__main__":
    execute = 4
    prayTimes = PrayTimes()
    prayTimes.setMethod('Makkah')
    prayers = ['Fajr', 'Sunrise', 'Dhuhr', 'Asr', 'Maghrib', 'Isha', 'Midnight', 'Fasting']
    prayers_lower = [prayer.lower() for prayer in prayers]
    if execute == 1:
        msg = 'Prayer Times for today in Riyadh/Saudi Arabia'
        print(msg,'\n'+ ('='* len(msg)))
        times = prayTimes.getTimes(date.today(), (24.68773,46.72185), 3, is_formatted=False);

        for prayer in prayers:
            print(f'{prayer:8}: {times[prayer.lower()]}')
    elif execute == 2:
        msg = 'Prayer Times for July 2019 in Riyadh/Saudi Arabia'
        print(msg, '\n' + ('=' * len(msg)))
        print(f'{"Day":4}', end='')
        for prayer in prayers:
            print(f'{prayer:10}', end='')
        for d in range(1, 32):
            times = prayTimes.getTimes(date(2019,7,d), (24.68773, 46.72185), 3, is_formatted=True)
            print()
            print(f'{d:3}  ', end='')
            for prayer in prayers_lower:
                print(f'{times[prayer]:10}', end='')
    elif execute ==3:
        msg = 'Prayer Times for one year in Riyadh/Saudi Arabia'
        daterange = pd.date_range(date(2019,1,1), date(2019,12,31))
        print(f'{"Date":8}', end='')
        for prayer in prayers:
            print(f'{prayer:10}', end='')
        for single_date in daterange:
            # print(single_date)
            # print(single_date.strftime("%Y-%m-%d"))
            # print(single_date.strftime("%b%d"))
            #times = prayTimes.getTimes(single_date, (24.68773, 46.72185), 3, is_formatted=True)
            times = prayTimes.getTimes(date(2019, single_date.month, single_date.day), (24.68773, 46.72185), 3, is_formatted=True)
            print()
            print(f'{single_date.strftime("%b%d"):8}  ', end='')
            for prayer in prayers_lower:
                print(f'{times[prayer]:10}', end='')
    elif execute == 4:
        msg = 'Prayer Times for one year in Riyadh/Saudi Arabia using Pandas'
        daterange = pd.date_range(date(2019, 1, 1), date(2019, 12, 31))
        titles=['Date']
        for prayer in prayers:
            titles.append(prayer)
        prayer_data=[]
        for single_date in daterange:

            times = prayTimes.getTimes(date(2019, single_date.month, single_date.day), (24.68773, 46.72185), 3,
                                       is_formatted=True)
            times_list = list(times.values())
            times_list=times_list[1:5]+times_list[6:]
            fasting_duration = times_list[-1]
            fasting_duration = fasting_duration.split(':')
            fasting_duration = int(fasting_duration[0])+int(fasting_duration[1])/60
            data_row = [single_date.strftime("%b%d")]+times_list[:-1]+[fasting_duration]
            prayer_data.append(data_row)
        df=pd.DataFrame(data=prayer_data,columns=titles)
        display(df)

,Date,Fajr,Sunrise,Dhuhr,Asr,Maghrib,Isha,Midnight,Fasting
0,Jan01,05:14,06:37,11:56,14:56,17:16,18:46,23:57,12.033333
1,Jan02,05:14,06:38,11:57,14:56,17:16,18:46,23:57,12.033333
2,Jan03,05:15,06:38,11:57,14:57,17:17,18:47,23:57,12.033333
3,Jan04,05:15,06:38,11:58,14:58,17:18,18:48,23:58,12.050000
4,Jan05,05:15,06:38,11:58,14:58,17:18,18:48,23:58,12.050000
5,Jan06,05:15,06:39,11:59,14:59,17:19,18:49,23:59,12.066667
6,Jan07,05:16,06:39,11:59,15:00,17:20,18:50,23:59,12.066667
7,Jan08,05:16,06:39,12:00,15:00,17:20,18:50,00:00,12.083333
8,Jan09,05:16,06:39,12:00,15:01,17:21,18:51,00:00,12.083333
9,Jan10,05:16,06:39,12:00,15:02,17:22,18:52,00:01,12.100000


In [5]:
prayTimes = PrayTimes()
prayTimes.setMethod('Makkah')
prayers = ['Fajr', 'Sunrise', 'Dhuhr', 'Asr', 'Maghrib', 'Isha', 'Midnight', 'Fasting']
prayers_lower = [prayer.lower() for prayer in prayers]
daterange = pd.date_range(date(2019, 1, 1), date(2019, 12, 31))
titles=['Date']
for prayer in prayers:
    titles.append(prayer)
prayer_data=[]
for single_date in daterange:

    times = prayTimes.getTimes(date(2019, single_date.month, single_date.day), (24.68773, 46.72185), 3,
                               is_formatted=True)
    times_list = list(times.values())
    times_list=times_list[1:5]+times_list[6:]
    fasting_duration = times_list[-1]
    fasting_duration = fasting_duration.split(':')
    fasting_duration = int(fasting_duration[0])+int(fasting_duration[1])/60
    data_row = [single_date.strftime("%Y%b%d")]+times_list[:-1]+[fasting_duration]
    prayer_data.append(data_row)
df=pd.DataFrame(data=prayer_data,columns=titles)
df.Date = pd.to_datetime(df['Date'], format='%Y%b%d')
display(df)

,Date,Fajr,Sunrise,Dhuhr,Asr,Maghrib,Isha,Midnight,Fasting
0,2019-01-01,05:14,06:37,11:56,14:56,17:16,18:46,23:57,12.033333
1,2019-01-02,05:14,06:38,11:57,14:56,17:16,18:46,23:57,12.033333
2,2019-01-03,05:15,06:38,11:57,14:57,17:17,18:47,23:57,12.033333
3,2019-01-04,05:15,06:38,11:58,14:58,17:18,18:48,23:58,12.050000
4,2019-01-05,05:15,06:38,11:58,14:58,17:18,18:48,23:58,12.050000
5,2019-01-06,05:15,06:39,11:59,14:59,17:19,18:49,23:59,12.066667
6,2019-01-07,05:16,06:39,11:59,15:00,17:20,18:50,23:59,12.066667
7,2019-01-08,05:16,06:39,12:00,15:00,17:20,18:50,00:00,12.083333
8,2019-01-09,05:16,06:39,12:00,15:01,17:21,18:51,00:00,12.083333
9,2019-01-10,05:16,06:39,12:00,15:02,17:22,18:52,00:01,12.100000


# Small Dataset of cities (702 city)

In [3]:
# Reading the cities df
cdf = pd.read_csv('..\Datasets\Cities2.csv')
cdf.head()

,id,country,city_name,lat,lng,alt,population,utc,mazhab
0,3041563,AD,Andorra la Vella,42.50779,1.52109,1037,20430,1.0,1
1,292223,AE,Dubai,25.06570,55.17128,3,1137347,4.0,4
2,1138958,AF,Kabul,34.52813,69.17233,1798,3043532,4.5,5
3,1138336,AF,Kandahar,31.61332,65.71013,1007,391190,4.5,5
4,1133616,AF,Mazar-e Sharif,36.70904,67.11087,361,303282,4.5,5


In [4]:
cdf.iloc[1]
cdf.iloc[1]['lat']
cdf.shape[0]
cdf.iloc[1].to_dict()

{'id': 292223,
 'country': 'AE',
 'city_name': 'Dubai',
 'lat': 25.0657,
 'lng': 55.17128,
 'alt': 3,
 'population': 1137347,
 'utc': 4.0,
 'mazhab': 4}

In [ ]:
prayTimes = PrayTimes()
prayTimes.setMethod('Makkah')
prayers = ['Fajr', 'Sunrise', 'Dhuhr', 'Asr', 'Maghrib', 'Isha', 'Midnight', 'Fasting']
mazaheb = {1:'MWL',2:'ISNA',3:'Egypt',4:'Makkah',5:'Karachi',6:'Tehran',7:'Jafari',8:'Kuwait',9:'France'}
prayers_lower = [prayer.lower() for prayer in prayers]
daterange = pd.date_range(date(2019, 1, 1), date(2019, 12, 31))
titles=['citi_id', 'Date']
for prayer in prayers:
    titles.append(prayer)
failed=[] 
full_df = []
for num in range(cdf.shape[0]):#3):#
    
    city = cdf.iloc[num]
    print(num+1, city['country'], city['city_name'], end='>>>')
    try:
        prayer_data=[]
        prayTimes.setMethod(mazaheb[city['mazhab']])
        for single_date in daterange:
            times = prayTimes.getTimes(date(2019, single_date.month, single_date.day), 
                                       (city['lat'], city['lng']), city['utc'],
                                       is_formatted=True)
            #print(single_date, times)
            times_list = list(times.values())
            times_list=times_list[1:5]+times_list[6:]
            fasting_duration = times_list[-1]
            fasting_duration = fasting_duration.split(':')
            fasting_duration = int(fasting_duration[0])+int(fasting_duration[1])/60
            #print(city['id'], times_list, fasting_duration)
            data_row = [city['id']] + [single_date.strftime("%Y%b%d")]+times_list[:-1]+[fasting_duration]
            #print(data_row)
            prayer_data.append(data_row)
        
        df=pd.DataFrame(data=prayer_data,columns=titles)
        df.Date = pd.to_datetime(df['Date'], format='%Y%b%d')
#         display(df.head())
        
        if type(full_df)==list:
            full_df = df.copy()
        else:
            full_df = pd.concat([full_df, df])
        print('OK')
    except:
        print('FAIL ', city['id'])
        failed.append(city['id'])
print('*** Done ***')      


In [21]:
#Checking the result
full_df.shape

(262070, 10)

In [22]:
# Saving the result
full_df.to_csv('..\Datasets\PrayerAndFasting.csv')

In [23]:
failed

[3423146, 2729907]

In [25]:
cdf[cdf.id.isin(failed)]

,id,country,city_name,lat,lng,alt,population,utc,mazhab
249,3423146,GL,Ilulissat,69.21667,-51.10000,11,4413,-3.0,9
569,2729907,SJ,Longyearbyen,78.22334,15.64689,1,2060,1.0,1


In [26]:
full_df.describe()

,citi_id,Fasting
count,2.620700e+05,262070.000000
mean,2.227132e+06,13.629209
std,1.577801e+06,1.782563
min,5.365400e+04,6.583333
25%,9.653010e+05,12.766667
50%,2.176770e+06,13.350000
75%,3.382160e+06,14.216667
max,1.063018e+07,22.583333


# Full dataset of cities (145k city)

In [4]:
fdf = pd.read_csv('..\Datasets\AllCities.csv')
fdf.head()

,id,country,city_name,lat,lng,alt,population,utc,mazhab
0,2960,SY,`Ayn Halaqim,34.93998,36.32212,596,0,2.0,3
1,4273,SY,Al Karamah,35.86865,39.27803,244,0,2.0,3
2,5174,SY,`Ariqah,32.88981,36.48336,804,0,2.0,3
3,9904,IR,Shahrak,36.17449,50.76908,1823,0,3.5,6
4,10570,IR,Alvand,36.18930,50.06430,1243,0,3.5,6


In [5]:
fdf.shape

(145658, 9)

In [7]:
prayTimes = PrayTimes()
prayTimes.setMethod('Makkah')
prayers = ['Fajr', 'Sunrise', 'Dhuhr', 'Asr', 'Maghrib', 'Isha', 'Midnight', 'Fasting']
mazaheb = {1:'MWL',2:'ISNA',3:'Egypt',4:'Makkah',5:'Karachi',6:'Tehran',7:'Jafari',8:'Kuwait',9:'France'}
prayers_lower = [prayer.lower() for prayer in prayers]
daterange = pd.date_range(date(2019, 1, 1), date(2019, 12, 31))
titles=['citi_id', 'Date', 'Fasting']


failed=[] 
full_df = []
for num in range(fdf.shape[0]):#3):#
    
    city = fdf.iloc[num]
    if num%100==0:
        print (int(num/100), end='')
    elif num%20==0:
        print ('.', end='')
        
#     print(num+1, city['country'], city['city_name'], end='>>>')
    try:
        prayer_data=[]
        prayTimes.setMethod(mazaheb[city['mazhab']])
        for single_date in daterange:
            times = prayTimes.getTimes(date(2019, single_date.month, single_date.day), 
                                       (city['lat'], city['lng']), city['utc'],
                                       is_formatted=False)

            fasting_duration = times['fasting']
            #print(city['id'], times_list, fasting_duration)
            data_row = [city['id']] + [single_date.strftime("%Y%b%d")] + [fasting_duration]
            #print(data_row)
            prayer_data.append(data_row)
        
        df=pd.DataFrame(data=prayer_data,columns=titles)
        df.Date = pd.to_datetime(df['Date'], format='%Y%b%d')
#         display(df.head())
        
        if type(full_df)==list:
            full_df = df.copy()
        else:
            full_df = pd.concat([full_df, df])
            
        if num % 2000==0:
            full_df.to_csv(f'..\Datasets\FullFastingDataset{int(num/2000)+1:04d}.csv')
            full_df=[]
#         print('OK')
    except:
#         print('FAIL ', city['id'])
        failed.append(city['id'])
    
full_df.to_csv(f'..\Datasets\FullFastingDataset{int(num/2000)+2:04d}.csv')
print('*** Done ***') 

0....1....2....3....4....5....6....7....8....9....10....11....12....13....14....15....16....17....18....19....20....21....22....23....24....25....26....27....28....29....30....31....32....33....34....35....36....37....38....39....40....41....42....43....44....45....46....47....48....49....50....51....52....53....54....55....56....57....58....59....60....61....62....63....64....65....66....67....68....69....70....71....72....73....74....75....76....77....78....79....80....81....82....83....84....85....86....87....88....89....90....91....92....93....94....95....96....97....98....99....100....101....102....103....104....105....106....107....108....109....110....111....112....113....114....115....116....117....118....119....120....121....122....123....124....125....126....127....128....129....130....131....132....133....134....135....136....137....138....139....140....141....142....143....144....145....146....147....148....149....150....151....152....153....154....155....156....157....158.

.1163....1164....1165....1166....1167....1168....1169....1170....1171....1172....1173....1174....1175....1176....1177....1178....1179....1180....1181....1182....1183....1184....1185....1186....1187....1188....1189....1190....1191....1192....1193....1194....1195....1196....1197....1198....1199....1200....1201....1202....1203....1204....1205....1206....1207....1208....1209....1210....1211....1212....1213....1214....1215....1216....1217....1218....1219....1220....1221....1222....1223....1224....1225....1226....1227....1228....1229....1230....1231....1232....1233....1234....1235....1236....1237....1238....1239....1240....1241....1242....1243....1244....1245....1246....1247....1248....1249....1250....1251....1252....1253....1254....1255....1256....1257....1258....1259....1260....1261....1262....1263....1264....1265....1266....1267....1268....1269....1270....1271....1272....1273....1274....1275....1276....1277....1278....1279....1280....1281....1282....1283....1284....1285....1286....1287...

# executed in 14h 57m 43s, finished 17:13:09 2019-07-22
'''0....1....2....3....4....5....6....7........1450....1451....1452....1453....1454....1455....1456..*** Done ***
'''


In [9]:
display(len(failed), failed)

0

[]

In [ ]:
# Saving the result
full_df.to_csv('..\Datasets\FullFastingDataset.csv')

In [ ]:
full_df.describe()

In [10]:
full_df.head()

,citi_id,Date,Fasting
0,8948837,2019-01-01,10.9454
1,8948837,2019-01-02,10.9563
2,8948837,2019-01-03,10.9682
3,8948837,2019-01-04,10.981
4,8948837,2019-01-05,10.9947
